# Лабораторная №3. Обработка последовательностей

**Выполнил**: Подцепко Игорь, учeбная группа M33351

## Предобработка текста

In [1]:
import re

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import tensorflow as tf
import nltk

from tqdm import tqdm
from pathlib import Path
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
from typing import Union

In [6]:
class Config:
    FILE_PATH: str = "data/moris.txt"
    STOP_CHAR: str = "."
    PREDICTION_LENGTH_LIMIT: int = 50
    EXAMPLE_PREFIX: str = "морис ска"
    BATCH_SIZE: int = len(EXAMPLE_PREFIX)

Поскольку для предобработки текста я собираюсь испольовать библиотеку `nltk` - необходимо кое-что загрузить:

In [7]:
nltk.download("punkt");

[nltk_data] Downloading package punkt to /home/igor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
def is_russian(word: str) -> bool:
    """ Allows only words consisting of Russian letters """
    pattern = re.compile("^[а-яА-Я]+$")
    return pattern.search(word) is not None


sentences = [
    " ".join([word for word in word_tokenize(sentence) if is_russian(word)]) + '.'
    for sentence in sent_tokenize(Path(Config.FILE_PATH).read_text().lower().replace("ё", "е"))
]
text = " ".join(sentences)
print(f'Found {len(sentences)} sentences, the length of the text is {len(text)} characters')

Found 6527 sentences, the length of the text is 308794 characters


## Модель LSTM

Далее закодируем все предложения для работы с моделью LSTM.

In [9]:
alphabet = sorted(list(set(text)))
alphabet_size = len(alphabet)

print(f"Alphabet: '{''.join(alphabet)}' (size {alphabet_size})")

codes = {symbol: code for (code, symbol) in enumerate(alphabet)}


def encode(char: str) -> list[int]:
    code = codes[char]
    return [1 if i == code else 0 for i in range(alphabet_size)]

print("Encoding:")
encoded = [[encode(char) for char in sentence] for sentence in tqdm(sentences)]

Alphabet: ' .абвгдежзийклмнопрстуфхцчшщъыьэюя' (size 34)
Encoding:


100%|██████████| 6527/6527 [00:00<00:00, 7845.75it/s]


Подготовим данные для обучения разделив предложения с помощью "скользящих окон" фиксированной длины (например, предложение "морис сказал" и окна длины 5 будем создавать пары вида ("мори", "c"), ("орис", " ") и так далее). На этих данных будем обучать модель LSTM ниже.

In [10]:
def split_dataset(encoded_sentences, batch_size) -> tuple[np.ndarray, np.ndarray]:
    prefixes = []
    continuations = []
    print("Split dataset to prefixes and continuations:")
    for sentence in tqdm(encoded_sentences):
        batch_count = len(sentence) - batch_size
        if batch_count <= 0:
            continue  # is too small
        for i in range(batch_count):
            prefixes.append(sentence[i: i + batch_size])
            continuations.append(sentence[i + batch_size])
    return np.array(prefixes), np.array(continuations)

In [11]:
phrase_prefixes, phrase_continuations = split_dataset(encoded, batch_size=Config.BATCH_SIZE)

Split dataset to prefixes and continuations:


100%|██████████| 6527/6527 [00:00<00:00, 16059.57it/s]


Следующая вспомогательная функция загружает модель LSTM, если она уже была обучена, или создает и обучает новую.

In [12]:
def load_lstm(
    prefixes: np.ndarray,
    continuations: np.ndarray,
    units: int,
    batch_size: int,
    alphabet_size: int,
    dropout: float,
    dense_activation: str,
    loss: str,
    optimizer: str,
    epoch_count: int,
    saving_path: Union[str, None] = None,
):
    """Creates and trains an LSTM model"""

    if saving_path is not None and os.path.exists(saving_path):
        return tf.keras.models.load_model(saving_path)

    model = tf.keras.models.Sequential()
    model.add(
        tf.keras.layers.LSTM(
            units, input_shape=(batch_size, alphabet_size), dropout=dropout
        )
    )  # LSTM cell
    model.add(
        tf.keras.layers.Dense(alphabet_size, activation=dense_activation)
    )  # Just your regular densely-connected NN layer.

    model.compile(loss=loss, optimizer=optimizer)

    model.fit(prefixes, continuations, epochs=epoch_count)

    if saving_path is not None:
        model.save(saving_path)

    return model


С помощью вышеупомянутой функции получим модель LSTM:

In [3]:
print("hello +", tf.config.list_physical_devices())

hello + [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [3]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'; import tensorflow as tf; print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [15]:
lstm = load_lstm(
    prefixes=phrase_prefixes,
    continuations=phrase_continuations,
    units=128, # Positive integer, dimensionality of the output space
    batch_size = Config.BATCH_SIZE,
    alphabet_size = alphabet_size,
    dropout = 0.2, # Fraction of the units to drop for the linear transformation of the inputs
    dense_activation="sigmoid", 
    loss="categorical_crossentropy", # for example, MSE or CrossEntropy
    optimizer="adam", 
    epoch_count=6,
    # saving_path="/drive/MyDrive/machine-learning/the-amazing-maurice-lstm.ps"
)

Epoch 1/6
3828/7649 [==============>...............] - ETA: 27s - loss: 2.6052

In [32]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

model = tf.keras.models.Sequential()

model.add(
    tf.keras.layers.LSTM(
        128, input_shape=(Config.BATCH_SIZE, alphabet_size), dropout=0.1
    )
)  # LSTM cell


2023-04-22 19:46:53.824272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-22 19:46:53.825590: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-22 19:46:53.826484: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Напишем также вспомогательную функцию, которая поможет предсказывать целые предложения с помощью обученной модели (а не по одному символу).

In [ ]:
%%capture --no-stdout


def predict_sentence(
    model,
    prefix,
    end_char: str = Config.STOP_CHAR,
    prediction_length_limit: int = Config.PREDICTION_LENGTH_LIMIT,
) -> str:
    """Predicts a sentence by prefix"""
    window = [encode(char) for char in prefix]
    while len(prefix) < prediction_length_limit:
        (prediction,) = model.predict(np.array([window]), verbose=0)
        prediction = alphabet[np.argmax(prediction)]
        prefix += prediction
        if prediction == end_char:
            break
        window = window[1:] + [encode(prediction)]
    return prefix


In [ ]:
print(f"LSTM prediction: {predict_sentence(lstm, prefix=Config.EXAMPLE_PREFIX)}")

LSTM prediction: морис сказал он.


## Марковская цепь

В следующей ячейке реализована Марковская цепь (реализация тривиальна).

In [ ]:
class MarkovChain:
    """ A class implementing a Markov chain """

    def __init__(self, batch_size: int, end_char: str = Config.STOP_CHAR, default_prediction_length_limit: int = Config.PREDICTION_LENGTH_LIMIT):
        """ Сreates an untrained Markov chain """
        self.transitions = defaultdict(lambda: defaultdict(lambda: 0))
        # transitions['префик']['с'] - кол-во вхождений 'префикс'
        self.batch_size = batch_size
        self.end_char = end_char
        self.default_prediction_length_limit = default_prediction_length_limit

    def fit(self, data: str):
        """ Fits the Markov chain on the provided text """
        for i in range(len(data) - self.batch_size - 1):
            self._fit(data[i : i + self.batch_size + 1])
    
    def _fit(self, data: str):
        """ Accepts a string of size batch_size + 1 and saves statistics
        to the frequency dictionary information about the transition to
        the last symbol """
        self.transitions[data[:-1]][data[-1]] += 1

    def predict(self, prefix: str, prediction_length_limit: Union[int, None] = None) -> str:
        """ Predict sentency by prefix """
        prediction = prefix
        prefix = prefix[-self.batch_size:]
        if prediction_length_limit is None:
            prediction_length_limit = self.default_prediction_length_limit
        while len(prediction) < prediction_length_limit:
            predicted_continuation = self._predict(prefix)
            prediction += predicted_continuation
            if predicted_continuation == self.end_char:
                break
            prefix = prefix[1:] + predicted_continuation
        return prediction

    def _predict(self, prefix: str) -> str:
        """ Predict continuation (1 char) by prefix """
        if prefix not in self.transitions:
            return self.end_char
        best_transition, _ = max(self.transitions[prefix].items(), key=lambda item: item[1])
        return best_transition

Создаем и обучаем марковскую цепь на той же книге "Удивительный Морис".

In [ ]:
markov_chain = MarkovChain(batch_size=Config.BATCH_SIZE)
markov_chain.fit(data=text)

In [ ]:
print(f"Markov chain prediction: {markov_chain.predict(prefix=Config.EXAMPLE_PREFIX)}")

Markov chain prediction: морис сказал сардины.


## Финальное сравнение моделей

In [ ]:
for sample in (
    "сказал морис",
    "крысолов номер",
    "опасный боб",
    "персик поднесла",
    "подумал морис",
    "морис подумал",
    "бургомистр",
    "странно поду",
    "мельком взглянув на",
    "машинное обучение",
):
    print(
        f'"{predict_sentence(lstm, prefix=sample)}" vs "{markov_chain.predict(prefix=sample)}"'
    )


"сказал морис." vs "сказал морис."
"крысолов номер они после тебе посмотрел на сторону" vs "крысолов номер один повернулся к персик которая по"
"опасный боб." vs "опасный боб."
"персик поднесла как будто то они была бы они была " vs "персик поднесла спичку к свече."
"подумал морис." vs "подумал морис."
"морис подумал морис." vs "морис подумал морис."
"бургомистр подомно ответил мальчик." vs "бургомистр."
"странно подумал морис." vs "странно подумал морис."
"мельком взглянув на мориса." vs "мельком взглянув на мориса."
"машинное обучение подумал морис." vs "машинное обучение."
